## Анализ пользовательского поведения в мобильном приложении

Оглавление :  

1. [Шаг 1. Загрузка данных и подготовка их к анализу](#step1) 


2. [Шаг 2. Изучение данных](#step2) 

    
3. [Шаг 3. Изучение воронки событий](#step3)

    
4. [Шаг 4. Изучение результатов эксперимента](#step4)


### Шаг 1. Загрузка данных и подготовка их к анализу <a id="step1"></a> 

In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import math as mth
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('/datasets/logs_exp.csv', delimiter='\t')

FileNotFoundError: [Errno 2] File /datasets/logs_exp.csv does not exist: '/datasets/logs_exp.csv'

In [ ]:
data.sample(10)

Заменим названия столбцов:
- EventName —> event_name  (название события);
- DeviceIDHash —> device_id (уникальный идентификатор пользователя);
- EventTimestamp —> event_time (время события);
- ExpId —> exp_group (номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная).

In [ ]:
data.rename(columns={'EventName': 'event_name', 'DeviceIDHash': 'device_id',
                     'EventTimestamp': 'timestamp', 'ExpId': 'exp_group'}, inplace=True)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['event_date'] = pd.to_datetime(data['timestamp'], unit='s')

In [ ]:
data['event_day'] = data['event_date'].astype('datetime64[D]')

In [ ]:
data['event_name'].unique()

In [ ]:
data['exp_group'].unique()

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
data

### Вывод
Загрузили данные, удалили полные дубликаты (один пользователь в одно и то же время не мог совершать 2 одинаковых события) и добавили столбцы с датой и временем в удобном формате.

### Шаг 2.  Изучение данных <a id="step2"></a> 

####  *Проверим, сколько всего событий в логе* 

In [ ]:
all_event = data['event_name'].count()
all_event

####  *Пользователей в логе:* 

In [ ]:
all_id = data['device_id'].nunique()
all_id

#### *Сколько в среднем событий приходится на пользователя* 

In [ ]:
plt.title('Распределение количества событий по количеству пользователей')
plt.xlabel('Количество событий')
plt.ylabel('Количество пользователей')
data['device_id'].value_counts().hist(bins=100, range=(0,300), figsize=(16,8), color = '#5352ed')
plt.show();

In [ ]:
mode_event_per_id = data['device_id'].value_counts().mode()[0]
mode_event_per_id

In [ ]:
mean_event_per_id = all_event/all_id
mean_event_per_id

####  *Проверим, данные за какой период доступны* 

In [ ]:
firts_date = data['event_date'].min()
last_date = data['event_date'].max()

In [ ]:
firts_date

In [ ]:
last_date

In [ ]:
time_to_event = data.groupby('event_day')['event_date'].count()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=time_to_event.index, y=time_to_event.values,
            text=time_to_event.values.round(2),
            textposition='outside'))
fig.update_layout(
    title='График логов по дням',
    xaxis_title='Дни',
    yaxis_title='Количество событий',
    title_x = 0.5,
    margin=dict(l=50, r=50, t=130, b=50))
fig.update_xaxes(nticks=20)
fig.show();

Полноценные данные есть только за август, очистим данные от логов июля:

In [ ]:
clean_data = data.query('event_date > "2019-08-01"')

In [ ]:
clean_event = clean_data['event_name'].count()
clean_event

In [ ]:
clean_id = clean_data['device_id'].nunique()
clean_id

In [ ]:
print('Потеряли: {} событий, {}%'
      .format(all_event - clean_event, round(100- clean_event/all_event*100, 2)))

In [ ]:
print('Потеряли: {} пользователей, {}%'
      .format(all_id - clean_id, round(100- clean_id/all_id*100, 2)))

#### *Убедимся, что есть пользователи из всех трёх экспериментальных групп:* 

In [ ]:
clean_data.groupby('exp_group').agg({'device_id':'nunique'})

### Вывод:
В массиве данных были неполные данные за июль, анализировать будем только данные за август (240887 событий от 7534 пользователей). Количество пользователей в группах распределено равномерно.

### Шаг 3. Изучение воронки событий <a id="step3"></a> 

####  *Изучим, какие события есть в логах и как часто они встречаются* 

In [ ]:
event_type_count = clean_data['event_name'].value_counts().reset_index()
event_type_count.columns = ['event_name','event_number']

In [ ]:
event_type_count

#### *Посчитаем долю пользователей, которые хоть раз совершали событие:* 

In [ ]:
event_users = clean_data.groupby('event_name').agg({'device_id':'nunique'}).sort_values(by='device_id', ascending=False).reset_index()

In [ ]:
event_users['%_of_id'] = event_users['device_id'] / clean_id * 100

In [ ]:
event_users

#### *Выстроим события последовательно:* 

Логика действий пользователя такая: он заходит на главный экран, смотрит или не смотрит обучающие картинки/видео, затем переходит к товарам. Далее начинается оформление и просмотр корзины, а после этого видит экран успешной оплаты.

Обучающие картинки/видео — совсем не обязательный этап, его можно скипнуть, поэтому в исследовании тоже не будет его учитывать.

In [ ]:
event_type_count_without_tutorial = event_type_count[:4]

In [ ]:
event_type_count_without_tutorial

In [ ]:
fig = go.Figure()
fig.add_trace(go.Funnel(y=event_type_count_without_tutorial['event_name'], x=event_type_count_without_tutorial['event_number']))
fig.update_layout(
    title='График воронки действий пользователей',
    title_x = 0.5,
    margin=dict(l=50, r=50, t=130, b=50))
fig.show()

#### *Посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем):*

In [ ]:
event_users_without_tutorial = event_users[:4]

In [ ]:
event_users_without_tutorial

In [ ]:
event_users_without_tutorial['%_of_initial'] = (event_users_without_tutorial['device_id']
                                              /event_users_without_tutorial['device_id'][0]) *100

In [ ]:
event_users_without_tutorial.loc[0, '%_of_previous'] = 100

In [ ]:
for i in range(1,4):
    event_users_without_tutorial['%_of_previous'][i] = (event_users_without_tutorial['device_id'][i]/
                                                        event_users_without_tutorial['device_id'][i-1]) *100

In [ ]:
event_users_without_tutorial

In [ ]:
fig = go.Figure()
fig.add_trace(go.Funnel(y=event_users_without_tutorial['event_name'], x=event_users_without_tutorial['device_id']))
fig.update_layout(
    title='График воронки пользователей',
    title_x = 0.5,
    margin=dict(l=50, r=50, t=130, b=50))
fig.show()

Проверим, сколько пользователей прошли все этапы в хронологическом порядке:

In [ ]:
users = clean_data.pivot_table(
    index='device_id', 
    columns='event_name', 
    values='event_date',
    aggfunc='min')

In [ ]:
users

In [ ]:
step_1 = ~users['MainScreenAppear'].isna()
step_2 = step_1 & (users['OffersScreenAppear'] > users['MainScreenAppear'])
step_3 = step_2 & (users['CartScreenAppear'] > users['OffersScreenAppear'])
step_4 = step_3 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

In [ ]:
n_pageview = users[step_1].shape[0]
n_offers = users[step_2].shape[0]
n_cart = users[step_3].shape[0]
n_payment = users[step_4].shape[0]

In [ ]:
event_users_without_tutorial['every_way'] = [n_pageview, n_offers, n_cart, n_payment]

In [ ]:
event_users_without_tutorial

In [ ]:
fig = go.Figure()
fig.add_trace(go.Funnel(y=event_users_without_tutorial['event_name'], x=event_users_without_tutorial['every_way']))
fig.update_layout(
    title='График воронки пользователей (тех, кто посетил каждый шаг в верной последовательности)',
    title_x = 0.5,
    margin=dict(l=50, r=50, t=130, b=50))
fig.show()

6.1% проходят через все этапы последовательно и 47.7% попадают от первого события к оплате (возможно, пропускают какой-то этап)

### Вывод
Пользователи часто просматривают главный экран, но к процессу выбора товаров доходят не все. Только 47.7% пользователей доходят от первого события до окна успешной оплаты.

Всего 454 пользователя прошли все этапы в хронологическом порядке, вероятно, большая часть пользователей не переходят к просмотру корзины, а выбирают быстрое оформление заказа.

### Шаг 4. Изучим результаты эксперимента <a id="step4"></a> 

In [ ]:
users_per_group = clean_data.groupby('exp_group').agg({'device_id':'nunique'}).reset_index()
users_per_group.columns = ['exp_group', 'users']

In [ ]:
users_per_group

####  *Проверим, находят ли статистические критерии разницу между выборками 246 и 247:*                                                                 

In [ ]:
data_table = clean_data.pivot_table(index='exp_group', columns='event_name', values='device_id', aggfunc='nunique').reset_index()

In [ ]:
data_table

In [ ]:
data_table = data_table.merge(users_per_group)

In [ ]:
unionAA = data_table.loc[0] + data_table.loc[1]
unionAA.name = 3
unionAA['exp_group'] = '246+247'

In [ ]:
data_table = data_table.append([unionAA])

In [ ]:
data_table

Определим нулевую и альтернативную гипотезы.

*Нулевая гипотеза*: между группами нет статистически значимых различий.

*Альтернативная гипотеза*: между группами есть статистически значимые различия.

In [ ]:
alpha = .01

In [ ]:
def stat_group (value_1, value_2, total_1, total_2):
    p1 = value_1/total_1
    p2 = value_2/total_2
    p_combined = (value_1 + value_2) / (total_1 + total_2)
    difference = p1 - p2 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/total_1 + 1/total_2))
    distr = st.norm(0, 1)  
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение: ', p_value)
    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными") 

In [ ]:
dict = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']

In [ ]:
def test(group_1, group_2):
    for i in range(4):
        stat_group(data_table[dict[i]][group_1],
                   data_table[dict[i]][group_2],
                   data_table['users'][group_1],
                   data_table['users'][group_2])

In [ ]:
test(0, 1)

Статистически значимых различий между группами A(246) и A(247) не обнаружено. Это значит, что тест проведён корректно и можно анализировать результаты A/B теста.

#### *Сравним результаты с каждой из контрольных групп в отдельности по каждому событию:*

Сравниваем A(246) с B:

In [ ]:
test(0, 2)

Сравниваем A(247) с B:

In [ ]:
test(1, 2)

Сравниваем AA(246+247) с B:

In [ ]:
test(3, 2)

####  *Обоснование уровня значимости* 

Выбрали уровень значимости в 0.01. В исследовании проверили 4 проверки для A/A теста и ещё 12 для A/B тестов.
Чтобы снизить групповую вероятность ошибки первого рода, можно использовать метод Шидака (он повысит мощность теста) или Бенферони. 

Ни в одной из проверок не было обнаружено статистически значимых различий, поэтому не будем менять значение alpha, но проверим, какое оно бы получилось с использованием метода Шидака:

In [ ]:
[1 - (1 - alpha)**(1/m) for m in range(1, 17)]

### Вывод
Статистически значимых различий между группами A (246 и 247) не нашли — это значит, что пользователи распределены в группах случайным образом. 

Также статистических различий не нашли и между этими группами и группой B (с измененным шрифтом), что означает, что шрифт не влияет на процесс покупки, поэтому можно смело менять шрифт во всём приложении, если это необходимо. Пользователей такая перемена не отпугивает :)